<a href="https://colab.research.google.com/github/Veeraa786/Analysis-of-Users-Logging-using-Python/blob/main/Prediction_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fetching Users Credentials**

**Importing the required libraries**

In [387]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Processing

In [388]:
takehome_users = pd.read_csv("/content/takehome_users.csv", encoding='latin-1')
takehome_user_engagement = pd.read_csv("/content/takehome_user_engagement.csv", encoding='latin-1')

#takehome_users.info()
takehome_users.describe().T

#takehome_user_engagement.info()

takehome_user_engagement.describe().T

takehome_user_engagement.user_id.nunique()

#print(takehome_users['last_session_creation_time'])

takehome_users['last_session_creation_time'] = pd.to_datetime(takehome_users['last_session_creation_time'], unit='s')

#print(takehome_users['last_session_creation_time'])
#takehome_users.last_session_creation_time.dtypes
#print(takehome_users.head(4))

takehome_users['last_session_creation_time'].min(),takehome_users['last_session_creation_time'].max()
df = takehome_user_engagement.copy()
df['date'] = pd.to_datetime(df.time_stamp)

#print(df['date'])

def rolling_count(df_group,frequency):
  return df_group.rolling(frequency, on='date')['user_id'].count()

df['visits_7_days'] = df.groupby('user_id', as_index=False, group_keys=False).apply(rolling_count, '7D')

#df.describe().T
#print(df['visits_7_days'])

df[df.visits_7_days >= 3.0]

,time_stamp,user_id,visited,date,visits_7_days
9,2014-02-09 03:45:04,2,1,2014-02-09 03:45:04,3.0
10,2014-02-13 03:45:04,2,1,2014-02-13 03:45:04,3.0
27,2013-02-19 22:08:03,10,1,2013-02-19 22:08:03,3.0
30,2013-03-02 22:08:03,10,1,2013-03-02 22:08:03,3.0
31,2013-03-05 22:08:03,10,1,2013-03-05 22:08:03,3.0
...,...,...,...,...,...
207898,2014-05-23 11:04:47,11988,1,2014-05-23 11:04:47,6.0
207899,2014-05-24 11:04:47,11988,1,2014-05-24 11:04:47,6.0
207900,2014-05-26 11:04:47,11988,1,2014-05-26 11:04:47,5.0
207901,2014-05-27 11:04:47,11988,1,2014-05-27 11:04:47,5.0


In [389]:
adopted = df.groupby('user_id')['visits_7_days'].max().reset_index()

adopted['adopted_user'] = adopted['visits_7_days'].apply(lambda x: 1 if x>=3 else 0)

adopted.head()


,user_id,visits_7_days,adopted_user
0,1,1.0,0
1,2,3.0,1
2,3,1.0,0
3,4,1.0,0
4,5,1.0,0


In [390]:
adopted.drop(['visits_7_days'], axis=1, inplace=True)
adopted.rename(columns={"user_id": "object_id"}, inplace=True)


In [391]:
adopted.set_index("object_id", inplace = True)

application_users = takehome_users.join(adopted, on = 'object_id', how='left')

application_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,0.0


In [392]:
application_users['last_session_creation_time'].fillna(0, inplace = True)

application_users['adopted_user'].fillna(0, inplace = True)

application_users.describe().T

,count,mean,std,min,25%,50%,75%,max
object_id,12000.0,6000.500000,3464.245950,1.0,3000.75,6000.5,9000.25,12000.0
opted_in_to_mailing_list,12000.0,0.249500,0.432742,0.0,0.00,0.0,0.00,1.0
enabled_for_marketing_drip,12000.0,0.149333,0.356432,0.0,0.00,0.0,0.00,1.0
org_id,12000.0,141.884583,124.056723,0.0,29.00,108.0,238.25,416.0
invited_by_user_id,6417.0,5962.957145,3383.761968,3.0,3058.00,5954.0,8817.00,11999.0
adopted_user,12000.0,0.133500,0.340128,0.0,0.00,0.0,0.00,1.0


In [393]:
application_users['email_domain'] = application_users.email.apply(lambda x: x.split('@')[1])
application_users['email_domain'].value_counts()


gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
gustr.com         1179
                  ... 
lrabg.com            1
mrytw.com            1
ugtav.com            1
hqhll.com            1
iuxiw.com            1
Name: email_domain, Length: 1184, dtype: int64

In [379]:
application_users[~application_users.invited_by_user_id.isnull()].creation_source.unique()


array(['GUEST_INVITE', 'ORG_INVITE'], dtype=object)

In [394]:
application_users.drop(['object_id', 'name', 'email', 'email_domain'], axis = 1, inplace = True)

